# Abundance of taxa on nose and severity

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.patches
from scipy.stats import pearsonr



In [2]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata_microbiome_type.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group,microbiome_type
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin


In [3]:
# Read in table at ASV level
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/feature_table_with_tax_labels_Genus.biom'
biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus-ASV-non-collapse.biom'
biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

# Get samples that don't start with 'nares' in metadata
nares_samples = metadata[metadata['group'].str.startswith('nares')].index

# Filter df to keep only nares samples
df = df.loc[nares_samples]
df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Cutibacterium_ASV-1,g___ASV-18,g___ASV-28,g___ASV-26,g__Cutibacterium_ASV-2,...,g__Copromonas_ASV-2,g__Microvirga_ASV-1,g__Leptotrichia_A_993758_ASV-19,g___ASV-104,g__Leptotrichia_A_993758_ASV-14,g__Capnocytophaga_820688_ASV-7,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-4,g__Bosea_ASV-2,g___ASV-140
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,51.0,0,193.0,0,0,0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
900223,3.0,0,20.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca010EBPN,83.0,0,49.0,0,81.0,19.0,0,0,0,10.0,...,0,0,0,0,0,0,0,0,0,0
900462,95.0,74.0,46.0,27.0,18.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
900053,184.0,0,0,0,0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,27.0,0,88.0,60.0,1.0,1.0,0,0,0,3.0,...,0,0,0,0,0,0,0,0,0,0
900402,13.0,0,0,0,0,4.0,0,0,0,1.0,...,0,0,0,0,0,0,0,0,0,0
9004022,4.0,0,11.0,0,12.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.columns.tolist()

['g__Streptococcus_ASV-1',
 'g__Streptococcus_ASV-2',
 'g__Corynebacterium_ASV-1',
 'g__Corynebacterium_ASV-3',
 'g___ASV-3',
 'g__Cutibacterium_ASV-1',
 'g___ASV-18',
 'g___ASV-28',
 'g___ASV-26',
 'g__Cutibacterium_ASV-2',
 'g__Bifidobacterium_388775_ASV-4',
 'g__Haemophilus_D_734546_ASV-1',
 'g__Dolosigranulum_ASV-1',
 'g__Dolosigranulum_ASV-2',
 'g__Haemophilus_D_734546_ASV-2',
 'g__Staphylococcus_ASV-2',
 'g__Staphylococcus_ASV-1',
 'g__Micrococcus_ASV-2',
 'g__Lactococcus_A_346120_ASV-1',
 'g__Limosilactobacillus_ASV-1',
 'g__Prevotella_ASV-1',
 'g__Corynebacterium_ASV-11',
 'g__Prevotella_ASV-2',
 'g__Fusobacterium_C_ASV-3',
 'g__Neisseria_563205_ASV-4',
 'g__Arthrobacter_D_ASV-1',
 'g__Gemella_ASV-2',
 'g___ASV-48',
 'g__Leptotrichia_A_993758_ASV-3',
 'g__Porphyromonas_A_859423_ASV-2',
 'g__Fusobacterium_C_ASV-2',
 'g___ASV-6',
 'g___ASV-61',
 'g__Brachybacterium_ASV-2',
 'g__Neisseria_563205_ASV-3',
 'g__Veillonella_A_ASV-4',
 'g__Sphingobacterium_ASV-5',
 'g__Jeotgalicoccus_A

In [5]:
# Calculate column sums and sort columns by sum in descending order
col_sums = df.sum()
df = df[col_sums.sort_values(ascending=False).index]
df.columns.tolist()

['g__Streptococcus_ASV-1',
 'g__Haemophilus_D_734546_ASV-1',
 'g__Streptococcus_ASV-2',
 'g__Corynebacterium_ASV-1',
 'g__Dolosigranulum_ASV-1',
 'g__Haemophilus_D_734546_ASV-2',
 'g__Staphylococcus_ASV-1',
 'g__Corynebacterium_ASV-3',
 'g___ASV-3',
 'g__Dolosigranulum_ASV-2',
 'g__Staphylococcus_ASV-2',
 'g__Streptococcus_ASV-3',
 'g___ASV-6',
 'g__SIO2C1_ASV-1',
 'g__Neisseria_563205_ASV-1',
 'g__Neisseria_563205_ASV-2',
 'g___ASV-16',
 'g__Veillonella_A_ASV-1',
 'g__Pseudomonas_E_647464_ASV-1',
 'g__Gemella_ASV-1',
 'g___ASV-1',
 'g__Cutibacterium_ASV-1',
 'g__Streptococcus_ASV-4',
 'g__Chryseobacterium_796614_ASV-1',
 'g__Granulicatella_ASV-1',
 'g__Corynebacterium_ASV-4',
 'g__Corynebacterium_ASV-2',
 'g__Porphyromonas_A_859423_ASV-1',
 'g__F0422_ASV-1',
 'g__Lactococcus_A_346120_ASV-1',
 'g__Granulicatella_ASV-2',
 'g__Cutibacterium_ASV-2',
 'g__Micrococcus_ASV-1',
 'g___ASV-22',
 'g__Prevotella_ASV-1',
 'g__Actinomyces_ASV-1',
 'g__SIO2C1_ASV-4',
 'g__Prevotella_ASV-4',
 'g__Aci

In [6]:
def rclr_transform(df, pseudocount=1e-6):
    """
    Applies Robust Centered Log-Ratio (RCLR) transformation to a DataFrame.
    Zeros are ignored in the geometric mean calculation per sample.
    """
    # Convert sparse to dense if needed
    df = df.sparse.to_dense()

    # Replace 0 with np.nan to ignore in log and mean
    df_masked = df.replace(0, np.nan)

    # Apply log (with pseudocount only where needed)
    log_df = np.log(df_masked + pseudocount)

    # Subtract mean of each row (feature)
    rclr_df = log_df.sub(log_df.mean(axis=1, skipna=True), axis=0)

    return rclr_df



rclr_df = rclr_transform(df)

In [7]:
# Map the 'group' column from metadata to df based on matching index
rclr_df['group'] = metadata.loc[rclr_df.index, 'group']
rclr_df

,g__Streptococcus_ASV-1,g__Haemophilus_D_734546_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Dolosigranulum_ASV-1,g__Haemophilus_D_734546_ASV-2,g__Staphylococcus_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Dolosigranulum_ASV-2,...,g___ASV-106,g__Parabacteroides_B_862066_ASV-1,g__Lactobacillus_ASV-2,g__Blautia_A_141781_ASV-3,g___ASV-116,g___ASV-108,g___ASV-154,g__Pseudomonas_E_650326_ASV-1,g__Massilia_ASV-6,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.109420,NaN,NaN,3.440285,2.048796,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,-1.146646,NaN,NaN,0.750474,-1.552111,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,2.327474,NaN,NaN,1.800454,1.044128,NaN,-1.398219,NaN,2.303083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,3.464733,1.801228,3.214922,2.739498,-1.089143,1.801228,-1.089143,2.206693,1.801228,-0.395996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,4.154583,3.630995,NaN,NaN,-1.060351,2.494996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.302349,NaN,NaN,3.483849,2.670073,NaN,0.392806,3.100856,-0.993487,2.051034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,1.246307,NaN,NaN,NaN,NaN,NaN,2.670342,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,0.130732,4.342859,NaN,1.142332,0.536197,2.178424,NaN,NaN,1.229344,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [8]:
rclr_df['group'].value_counts()

# rclr_df = rclr_df.rename(columns={
#     col: col.replace("g__Haemophilus_D_734546", "g__Haemophilus")
#     for col in df.columns
#     if col.startswith("g__Haemophilus_D_734546")
# })


# rclr_df = rclr_df.rename(columns={
#     col: col.replace("g__Veillonella_A", "g__Veillonella")
#     for col in df.columns
#     if col.startswith("g__Veillonella_A")
# })



group
nares-AD    108
nares-H      89
Name: count, dtype: int64

In [9]:
rclr_df.columns.tolist()

['g__Streptococcus_ASV-1',
 'g__Haemophilus_D_734546_ASV-1',
 'g__Streptococcus_ASV-2',
 'g__Corynebacterium_ASV-1',
 'g__Dolosigranulum_ASV-1',
 'g__Haemophilus_D_734546_ASV-2',
 'g__Staphylococcus_ASV-1',
 'g__Corynebacterium_ASV-3',
 'g___ASV-3',
 'g__Dolosigranulum_ASV-2',
 'g__Staphylococcus_ASV-2',
 'g__Streptococcus_ASV-3',
 'g___ASV-6',
 'g__SIO2C1_ASV-1',
 'g__Neisseria_563205_ASV-1',
 'g__Neisseria_563205_ASV-2',
 'g___ASV-16',
 'g__Veillonella_A_ASV-1',
 'g__Pseudomonas_E_647464_ASV-1',
 'g__Gemella_ASV-1',
 'g___ASV-1',
 'g__Cutibacterium_ASV-1',
 'g__Streptococcus_ASV-4',
 'g__Chryseobacterium_796614_ASV-1',
 'g__Granulicatella_ASV-1',
 'g__Corynebacterium_ASV-4',
 'g__Corynebacterium_ASV-2',
 'g__Porphyromonas_A_859423_ASV-1',
 'g__F0422_ASV-1',
 'g__Lactococcus_A_346120_ASV-1',
 'g__Granulicatella_ASV-2',
 'g__Cutibacterium_ASV-2',
 'g__Micrococcus_ASV-1',
 'g___ASV-22',
 'g__Prevotella_ASV-1',
 'g__Actinomyces_ASV-1',
 'g__SIO2C1_ASV-4',
 'g__Prevotella_ASV-4',
 'g__Aci

In [10]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()
        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df

In [11]:
# Filter for nares samples
nares_samples = rclr_df[rclr_df['group'].isin(['nares-AD', 'nares-H'])]
nares_samples

,g__Streptococcus_ASV-1,g__Haemophilus_D_734546_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Dolosigranulum_ASV-1,g__Haemophilus_D_734546_ASV-2,g__Staphylococcus_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Dolosigranulum_ASV-2,...,g___ASV-106,g__Parabacteroides_B_862066_ASV-1,g__Lactobacillus_ASV-2,g__Blautia_A_141781_ASV-3,g___ASV-116,g___ASV-108,g___ASV-154,g__Pseudomonas_E_650326_ASV-1,g__Massilia_ASV-6,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.109420,NaN,NaN,3.440285,2.048796,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,-1.146646,NaN,NaN,0.750474,-1.552111,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,2.327474,NaN,NaN,1.800454,1.044128,NaN,-1.398219,NaN,2.303083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,3.464733,1.801228,3.214922,2.739498,-1.089143,1.801228,-1.089143,2.206693,1.801228,-0.395996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,4.154583,3.630995,NaN,NaN,-1.060351,2.494996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.302349,NaN,NaN,3.483849,2.670073,NaN,0.392806,3.100856,-0.993487,2.051034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,1.246307,NaN,NaN,NaN,NaN,NaN,2.670342,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,0.130732,4.342859,NaN,1.142332,0.536197,2.178424,NaN,NaN,1.229344,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [12]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()

        # Skip comparison if either group has no data
        if len(x) == 0 or len(y) == 0:
            continue

        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    if len(raw_pvals) == 0:
        return pd.DataFrame(columns=['Comparison', 'Pair', 'Raw p-value', 'BH-corrected p-value'])

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df


In [13]:
# Define custom color palette
group_palette = {
    'nares-H': '#ADD8E6',     # baby blue
    'nares-AD': '#E31A1C'    # red
}

strip_palette = {
    'nares-H': '#6CA6CD',     # darker baby blue
    'nares-AD': '#A50000'    # darker red
}


In [14]:
def plot_multi_taxa_boxplots(data, taxa_list, group_palette, strip_palette, title_name, order=['nares-H', 'nares-AD']):
    n_taxa = len(taxa_list)
    group_count = len(order)

    # Prepare long format dataframe
    plot_data = []
    for i, taxon in enumerate(taxa_list):
        for j, group in enumerate(order):
            xpos = i * group_count + j
            values = data.loc[data['group'] == group, taxon]
            for v in values:
                plot_data.append({
                    'x': xpos,
                    'taxon': taxon.replace(' g__', '').split('_')[0],
                    'value': v,
                    'group': group
                })

    plot_df = pd.DataFrame(plot_data)

    # Plot
    fig, ax = plt.subplots(figsize=(n_taxa * 2.5, 5))

    sns.boxplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=group_palette,
        width=0.5,
        fliersize=0,
        dodge=False,
        ax=ax
    )

    sns.stripplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=strip_palette,
        dodge=False,
        jitter=True,
        size=4,
        alpha=0.75,
        ax=ax
    )

    # Remove duplicated legends
    handles, labels = ax.get_legend_handles_labels()

    # Count samples per group
    group_counts = {g: data[data['group'] == g].shape[0] for g in order}

    # Create updated legend labels
    legend_labels = [f"{g.replace('skin-', '')} (n={group_counts[g]})" for g in order]

    # Add updated legend — horizontally at bottom-left
    ax.legend(
        handles[:3],
        legend_labels,
        # title='Sample',
        loc='lower left',
        bbox_to_anchor=(0.35, -0.35),  # adjust y for spacing below plot
        ncol=len(order),            # show horizontally
        fontsize=12,
        title_fontsize=10,
        frameon=True,
        borderaxespad=0
    )


    # X-tick labels per taxon group
    xtick_positions = [i * group_count + 1 for i in range(n_taxa)]
    # xtick_labels = [taxon.replace('g__', '').split('_')[0] for taxon in taxa_list]
    xtick_labels = [taxon.replace('g__', '').replace('_ASV-', '\nASV-') for taxon in taxa_list]
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(xtick_labels, fontsize=14)

    ax.set_xlim(-0.5, n_taxa * group_count - 0.5)
    ax.set_ylabel("RCLR-transformed abundance", fontsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_xlabel(" ")

    ax.set_title(title_name, fontsize=18, pad=14)

    # ------------------------
    # Annotate p-values
    # ------------------------
    for i, taxon in enumerate(taxa_list):
        stats = pairwise_mannwhitney_bh(data, taxon)
        y_max = data[taxon].max()
        offset = y_max * 0.15

        for k, row in stats.iterrows():
            g1, g2 = row['Pair']
            pval = row['BH-corrected p-value']
            xpos1 = i * group_count + order.index(g1)
            xpos2 = i * group_count + order.index(g2)
            x = (xpos1 + xpos2) / 2
            y = y_max + offset * k

            stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
            label = f"{pval:.1e} {stars}"

            ax.plot([xpos1, xpos1, xpos2, xpos2], [y - 0.01, y, y, y - 0.01], lw=1, color='black')
            ax.text(x, y + 0.01, label, ha='center', va='bottom', fontsize=12)

    sns.despine()
    fig.tight_layout()
    return fig

In [15]:
taxa_list = ['g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Staphylococcus_ASV-1',
            'g__Micrococcus_ASV-1', 'g__Veillonella_A_ASV-1', 'g__Haemophilus_D_734546_ASV-1']
fig = plot_multi_taxa_boxplots(nares_samples, taxa_list, group_palette, strip_palette, 'Differential Taxa on Skin by Atopic Dermatitis Status')
fig.savefig('../Plots/Analysis_figures/RCLR_Abundance/rclr_multi_taxa_boxplot_all_taxa_ASV_nares.png', dpi=600)


/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


## Show correlation with lesion severity

In [16]:
# Map o_scorad from metadata to nares_samples based on matching indexes
nares_samples = nares_samples.merge(metadata[['o_scorad']], left_index=True, right_index=True)
nares_samples

,g__Streptococcus_ASV-1,g__Haemophilus_D_734546_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Dolosigranulum_ASV-1,g__Haemophilus_D_734546_ASV-2,g__Staphylococcus_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Dolosigranulum_ASV-2,...,g__Parabacteroides_B_862066_ASV-1,g__Lactobacillus_ASV-2,g__Blautia_A_141781_ASV-3,g___ASV-116,g___ASV-108,g___ASV-154,g__Pseudomonas_E_650326_ASV-1,g__Massilia_ASV-6,group,o_scorad
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.109420,NaN,NaN,3.440285,2.048796,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40
900223,-1.146646,NaN,NaN,0.750474,-1.552111,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34
Ca010EBPN,2.327474,NaN,NaN,1.800454,1.044128,NaN,-1.398219,NaN,2.303083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21
900462,3.464733,1.801228,3.214922,2.739498,-1.089143,1.801228,-1.089143,2.206693,1.801228,-0.395996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40
900053,4.154583,3.630995,NaN,NaN,-1.060351,2.494996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,2.302349,NaN,NaN,3.483849,2.670073,NaN,0.392806,3.100856,-0.993487,2.051034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54
900402,1.246307,NaN,NaN,NaN,NaN,NaN,2.670342,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,NaN
9004022,0.130732,4.342859,NaN,1.142332,0.536197,2.178424,NaN,NaN,1.229344,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38


In [17]:
# Convert 'o_scorad' to numeric (coerce errors to NaN)
rclr_df['o_scorad'] = pd.to_numeric(metadata['o_scorad'], errors='coerce')

# Drop rows with missing SCORAD values
rclr_df = rclr_df.dropna(subset=['o_scorad'])
rclr_df


,g__Streptococcus_ASV-1,g__Haemophilus_D_734546_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Dolosigranulum_ASV-1,g__Haemophilus_D_734546_ASV-2,g__Staphylococcus_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Dolosigranulum_ASV-2,...,g__Parabacteroides_B_862066_ASV-1,g__Lactobacillus_ASV-2,g__Blautia_A_141781_ASV-3,g___ASV-116,g___ASV-108,g___ASV-154,g__Pseudomonas_E_650326_ASV-1,g__Massilia_ASV-6,group,o_scorad
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.109420,NaN,NaN,3.440285,2.048796,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900223,-1.146646,NaN,NaN,0.750474,-1.552111,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34.0
Ca010EBPN,2.327474,NaN,NaN,1.800454,1.044128,NaN,-1.398219,NaN,2.303083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21.0
900462,3.464733,1.801228,3.214922,2.739498,-1.089143,1.801228,-1.089143,2.206693,1.801228,-0.395996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900053,4.154583,3.630995,NaN,NaN,-1.060351,2.494996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900399,-1.152529,NaN,-1.152529,NaN,NaN,NaN,3.401347,NaN,-0.459382,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9003992,2.302349,NaN,NaN,3.483849,2.670073,NaN,0.392806,3.100856,-0.993487,2.051034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9004022,0.130732,4.342859,NaN,1.142332,0.536197,2.178424,NaN,NaN,1.229344,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38.0


In [18]:
# Filter to only include skin-ADL samples for severity correlation analysis
rclr_df = rclr_df[rclr_df['group'] == 'nares-AD']
rclr_df

,g__Streptococcus_ASV-1,g__Haemophilus_D_734546_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Dolosigranulum_ASV-1,g__Haemophilus_D_734546_ASV-2,g__Staphylococcus_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Dolosigranulum_ASV-2,...,g__Parabacteroides_B_862066_ASV-1,g__Lactobacillus_ASV-2,g__Blautia_A_141781_ASV-3,g___ASV-116,g___ASV-108,g___ASV-154,g__Pseudomonas_E_650326_ASV-1,g__Massilia_ASV-6,group,o_scorad
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.109420,NaN,NaN,3.440285,2.048796,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900223,-1.146646,NaN,NaN,0.750474,-1.552111,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34.0
Ca010EBPN,2.327474,NaN,NaN,1.800454,1.044128,NaN,-1.398219,NaN,2.303083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21.0
900462,3.464733,1.801228,3.214922,2.739498,-1.089143,1.801228,-1.089143,2.206693,1.801228,-0.395996,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900053,4.154583,3.630995,NaN,NaN,-1.060351,2.494996,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900399,-1.152529,NaN,-1.152529,NaN,NaN,NaN,3.401347,NaN,-0.459382,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9003992,2.302349,NaN,NaN,3.483849,2.670073,NaN,0.392806,3.100856,-0.993487,2.051034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9004022,0.130732,4.342859,NaN,1.142332,0.536197,2.178424,NaN,NaN,1.229344,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38.0


In [19]:
fig, axes = plt.subplots(1, len(taxa_list), figsize=(len(taxa_list) * 2, 4), sharey=True)

for i, taxon in enumerate(taxa_list):
    ax = axes[i]
    
    # Drop missing values
    df = rclr_df[['o_scorad', taxon]].dropna()

    # Plot regression
    sns.regplot(
        data=df,
        x='o_scorad',
        y=taxon,
        scatter_kws={'alpha': 0.5, 's': 20},
        line_kws={'color': 'black'},
        ax=ax
    )

    # Compute Pearson correlation
    r, pval = pearsonr(df['o_scorad'], df[taxon])
    r_label = f"Pearson r = {r:.2f}\np = {pval:.1e}"
    ax.text(0.05, 0.95, r_label, transform=ax.transAxes,
            fontsize=10, va='top', ha='left', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # Format axes
    # ax.set_title(taxon.replace(' g__', '').split('_')[0], fontsize=12)
    ax.set_title(taxon.replace('g__', ''), fontsize=10)
    ax.set_xlabel("SCORAD Severity", fontsize=12)
    if i == 0:
        ax.set_ylabel("RCLR-transformed abundance", fontsize=12)
    else:
        ax.set_ylabel("")
        ax.set_yticklabels([])

    ax.set_ylim(-3, 7)

plt.tight_layout(rect=[0, 0, 1, 0.92])  # Leaves space for suptitle
plt.suptitle("Correlation Between Taxa on Nares and Atopic Dermatitis Severity", fontsize=14, y=0.98)
plt.savefig('../Plots/Analysis_figures/Severity_Correlations/rclr_abundance_vs_severity_ASV_nares.png', dpi=600)